In [ ]:
import numpy as np
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from scipy.stats import kurtosis
import numpy as np
import polars as pl
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

from sklearn.base import clone
from copy import deepcopy
import optuna
from scipy.optimize import minimize
import os
import matplotlib.pyplot as plt
import seaborn as sns

import re
from colorama import Fore, Style

from tqdm import tqdm
from IPython.display import clear_output
from concurrent.futures import ThreadPoolExecutor

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

import lightgbm as lgb
from catboost import CatBoostRegressor, CatBoostClassifier
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import *
from sklearn.metrics import *

SEED = 42
n_splits = 5

import numpy as np
import pandas as pd
import os
import re
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

from colorama import Fore, Style
from IPython.display import clear_output
import warnings
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None


def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)

    # Convert 'time_of_day' to datetime
    df['time_of_day'] = pd.to_datetime(df['time_of_day'])
    df.set_index('time_of_day', inplace=True)

    # Daily aggregation
    daily_features = df.resample('D').agg({
        'X': ['mean', 'std', 'max', 'min', 'median', 'skew', lambda x: kurtosis(x, nan_policy='omit')],
        'Y': ['mean', 'std', 'max', 'min', 'median', 'skew', lambda x: kurtosis(x, nan_policy='omit')],
        'Z': ['mean', 'std', 'max', 'min', 'median', 'skew', lambda x: kurtosis(x, nan_policy='omit')],
        'enmo': ['mean', 'std', 'max', 'min', 'median', 'skew', lambda x: kurtosis(x, nan_policy='omit')],
        'light': ['mean', 'std'],
        'battery_voltage': ['mean', 'std'],
        'non-wear_flag': 'sum'
    })

    # Flatten the MultiIndex columns
    daily_features.columns = ['_'.join(col).strip() for col in daily_features.columns.values]
    
    # Add additional features
    daily_features['activity_count'] = (df['enmo'] > 0.1).resample('D').sum()  # Adjust threshold as needed
    daily_features['days_active'] = (df['non-wear_flag'] == 0).resample('D').sum()  # Count active days
    
    # Rate of change features
    daily_features['enmo_change'] = daily_features['enmo_mean'].diff()
    
    # Rolling statistics
    rolling_windows = [5, 10, 15]  # Days
    for window in rolling_windows:
        daily_features[f'enmo_rolling_mean_{window}'] = daily_features['enmo_mean'].rolling(window=window).mean()
        daily_features[f'enmo_rolling_std_{window}'] = daily_features['enmo_std'].rolling(window=window).std()

    # Frequency domain features using FFT
    for axis in ['X', 'Y', 'Z', 'enmo']:
        freq_features = np.fft.fft(df[axis])
        daily_features[f'{axis}_dominant_freq'] = np.abs(freq_features).argmax()

    # Reset index to retain 'id'
    daily_features.reset_index(inplace=True)
    
    # Extract child ID from filename
    child_id = filename.split('=')[1]
    daily_features['id'] = child_id

    return daily_features



def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
    # Concatenate all results into a single DataFrame
    features_df = pd.concat(results, ignore_index=True)

    return features_df

# Build the autoencoder model
def build_autoencoder(input_dim, encoding_dim):
    input_layer = Input(shape=(input_dim,))
    
    # Encoder: compressing the input
    encoded = Dense(encoding_dim, activation='relu')(input_layer)
    
    # Decoder: reconstructing the input
    decoded = Dense(input_dim, activation='sigmoid')(encoded)
    
    # Autoencoder model
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    
    # Encoder model (for getting the compressed representation)
    encoder = Model(inputs=input_layer, outputs=encoded)
    
    autoencoder.compile(optimizer=Adam(), loss='mse')
    
    return autoencoder, encoder

# Function to perform dimensionality reduction using an autoencoder
def perform_autoencoder(df, encoding_dim=50, epochs=50, batch_size=32):
    """
    Perform dimensionality reduction using an Autoencoder.

    Parameters:
    df (pd.DataFrame): The input DataFrame with numerical features.
    encoding_dim (int): The dimension of the encoded space.
    epochs (int): Number of epochs to train the autoencoder.
    batch_size (int): Size of the batches for training.

    Returns:
    pd.DataFrame: DataFrame containing the reduced-dimensional representation (encoding).
    """
    
    # Step 1: Standardize the data
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    
    # Step 2: Build the autoencoder
    input_dim = df_scaled.shape[1]
    autoencoder, encoder = build_autoencoder(input_dim, encoding_dim)
    
    # Step 3: Train the autoencoder
    autoencoder.fit(df_scaled, df_scaled, epochs=epochs, batch_size=batch_size, shuffle=True, verbose=1)
    
    # Step 4: Get the encoded (reduced) representation
    encoded_data = encoder.predict(df_scaled)
    
    # Step 5: Create a DataFrame for the encoded features
    df_encoded = pd.DataFrame(encoded_data, columns=[f'Enc_{i+1}' for i in range(encoded_data.shape[1])])
    
    return df_encoded

# Load the data
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

# Columns to drop
columns_to_drop = [
    'enmo_change',
    'enmo_rolling_mean_5',
    'enmo_rolling_std_5',
    'enmo_rolling_mean_10',
    'enmo_rolling_std_10',
    'enmo_rolling_mean_15',
    'enmo_rolling_std_15'
]

# Drop the columns
train_ts.drop(columns=columns_to_drop, inplace=True)

# Drop the columns
test_ts.drop(columns=columns_to_drop, inplace=True)

# Drop 'id' column for training
df_train = train_ts.drop('id', axis=1)
df_test = test_ts.drop('id', axis=1)

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

def build_autoencoder(input_dim, encoding_dim):
    input_layer = Input(shape=(input_dim,))
    
    # Encoder: compressing the input
    encoded = Dense(encoding_dim, activation='relu')(input_layer)
    
    # Decoder: reconstructing the input
    decoded = Dense(input_dim, activation='sigmoid')(encoded)
    
    # Autoencoder model
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    
    # Encoder model (for getting the compressed representation)
    encoder = Model(inputs=input_layer, outputs=encoded)
    
    autoencoder.compile(optimizer=Adam(), loss='mse')
    
    return autoencoder, encoder

def perform_autoencoder(df, encoding_dim=50, epochs=200, batch_size=32):
    scaler = StandardScaler()
    
    # Exclude non-numeric columns
    df_numeric = df.select_dtypes(include=[np.number])
    
    # Check for and handle NaN values
    if df_numeric.isnull().values.any():
        df_numeric.fillna(df_numeric.mean(), inplace=True)

    if np.isinf(df_numeric).values.any():
        df_numeric.replace([np.inf, -np.inf], np.nan, inplace=True)
        df_numeric.fillna(df_numeric.mean(), inplace=True)

    df_scaled = scaler.fit_transform(df_numeric)

    # Check for NaN values after scaling
    if np.isnan(df_scaled).any() or np.isinf(df_scaled).any():
        print("NaN or infinite values detected in the scaled data!")
        return None  # or handle as needed

    input_dim = df_scaled.shape[1]
    autoencoder, encoder = build_autoencoder(input_dim, encoding_dim)
    
    # Use a smaller learning rate for stability
    autoencoder.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')
    
    # Set up early stopping
    early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

    # Train the autoencoder with early stopping
    autoencoder.fit(df_scaled, df_scaled, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    shuffle=True, 
                    verbose=1, 
                    callbacks=[early_stopping])

    encoded_data = encoder.predict(df_scaled)
    
    df_encoded = pd.DataFrame(encoded_data, columns=[f'Enc_{i+1}' for i in range(encoded_data.shape[1])])
    
    return df_encoded

# # Perform autoencoder dimensionality reduction
# train_ts_encoded = perform_autoencoder(df_train, encoding_dim=60, epochs=100, batch_size=32)
# test_ts_encoded = perform_autoencoder(df_test, encoding_dim=60, epochs=100, batch_size=32)

train_ts_encoded = perform_autoencoder(df_train, encoding_dim=64, epochs=256, batch_size=32)
test_ts_encoded = perform_autoencoder(df_test, encoding_dim=64,epochs=256, batch_size=32)

time_series_cols = train_ts_encoded.columns.tolist()
#time_series_cols.remove("id")

train_ts_encoded["id"]=train_ts["id"]

test_ts_encoded['id']=test_ts["id"]
#test_ts_pca["id"]=test_ts["id"]

#train = pd.merge(train, train_ts, how="left", on='id')
test = pd.merge(test, test_ts_encoded, how="left", on='id')
#test_ = pd.merge(test, test_ts, how="left", on='id')

train = pd.merge(train, train_ts_encoded, how="left", on='id')
#test = pd.merge(test, test_ts, how="inner", on='id')
analysis=train
analysis = analysis.dropna(subset='Enc_4')
analysis=analysis.dropna(subset='sii')
analysis = analysis.drop('id', axis=1)

def engineer_features(df):
    # Create interaction features
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    
    # Create categorical features
    #df['Age_Group'] = pd.cut(df['Basic_Demos-Age'], bins=[0, 12, 18, 25, 100], labels=['Child', 'Teen', 'Young Adult', 'Adult'])
    #df['BMI_Category'] = pd.cut(df['Physical-BMI'], bins=[0, 18.5, 25, 30, 100], labels=['Underweight', 'Normal', 'Overweight', 'Obese'])
    
    return df

test = engineer_features(test)

train=engineer_features(train)

#train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

nan_rows = train.query('sii.isna()', engine='python')

import pandas as pd
from sklearn.impute import KNNImputer

# Assuming 'train' is your DataFrame

# Step 1: Create the KNN imputer
imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors

# Step 2: Select numeric columns and fit the imputer
numeric_cols = train.select_dtypes(include=['float64', 'int64']).columns
imputed_data = imputer.fit_transform(train[numeric_cols])

# Step 3: Create a new DataFrame with the imputed values
train_imputed = pd.DataFrame(imputed_data, columns=numeric_cols)

# Step 4: Convert the 'sii' column back to integers
train_imputed['sii'] = train_imputed['sii'].round().astype(int)

# If there are other columns to retain, you can merge them back
for col in train.columns:
    if col not in numeric_cols:
        train_imputed[col] = train[col]

# Now, check if 'sii' has been filled and is of integer type
print(train_imputed['sii'].isna().sum())  # Should be 0 if all NaNs were filled
print(train_imputed['sii'].dtype)  # Should show 'int'

train_imputed['sii']=train['sii']

train=train_imputed

featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
                'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
                'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
                'PreInt_EduHx-computerinternet_hoursday', 'sii','BMI_Age','Internet_Hours_Age','BMI_Internet_Hours']

featuresCols += time_series_cols

train = train[featuresCols]
#test= test[featuresCols]
train = train.dropna(subset='sii')

cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 'Fitness_Endurance-Season', 
          'FGC-Season', 'BIA-Season', 'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']

def update(df):
    for c in cat_c: 
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
    return df

featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
                'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
                'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
                'PreInt_EduHx-computerinternet_hoursday','BMI_Age','Internet_Hours_Age','BMI_Internet_Hours']

featuresCols += time_series_cols

test= test[featuresCols]

train = update(train)
test = update(test)

def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
    return {value: idx for idx, value in enumerate(unique_values)}

"""This Mapping Works Fine For me I also Check Each Values in Train and test Using Logic. There no Data Lekage."""

for col in cat_c:
    mapping_train = create_mapping(col, train)
    mapping_test = create_mapping(col, test)
    
    train[col] = train[col].replace(mapping_train).astype(int)
    test[col] = test[col].replace(mapping_test).astype(int)

print(f'Train Shape : {train.shape} || Test Shape : {test.shape}')

analysis=train.dropna(subset='Enc_51')

missing_df=train[train['Enc_51'].isna()]

import pandas as pd

# Assuming analysis is your complete DataFrame (1000 data points)
# and missing_df is your DataFrame with missing values (3000 data points)

# Step 1: Store original data types
original_dtypes = analysis.dtypes.to_dict()

# Step 2: Split the complete data into features and target
X_complete = analysis.drop('sii', axis=1)
y_complete = analysis['sii']

# Step 3: Identify categorical and numerical columns
categorical_features = [
    'Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 
    'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
    'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season'
]

# Step 3.1: Drop categorical columns for mean calculation
numerical_df = analysis.drop(columns=categorical_features + ['sii'])

# Group by the target variable and calculate means for numerical features
mean_impute = numerical_df.groupby(analysis['sii']).mean()

# Group by the target variable and calculate modes for categorical features
mode_impute = analysis[categorical_features].groupby(analysis['sii']).agg(
    lambda x: x.mode()[0] if not x.mode().empty else None
)

# Step 4: Fill missing values class-wise
def fill_missing_values(row):
    class_value = row['sii']
    if pd.isnull(class_value):
        return row  # No action if class_value is NaN

    for col in row.index:
        if pd.isnull(row[col]):
            if col in mean_impute.columns and class_value in mean_impute.index:  # For numerical columns
                row[col] = mean_impute.loc[class_value, col]
            elif col in mode_impute.columns and class_value in mode_impute.index:  # For categorical columns
                row[col] = mode_impute.loc[class_value, col]
    return row

# Apply the filling function to each row in the missing_df
df_missing_filled = missing_df.apply(fill_missing_values, axis=1)

# Step 5: Combine datasets
final_dataset = pd.concat([analysis, df_missing_filled], ignore_index=True)

# Step 6: Convert categorical features to int64 safely
for col in categorical_features:
    # Check for unique values and their counts
    unique_values = final_dataset[col].unique()
    print(f'Unique values in {col}: {unique_values}')

    # Fill NaNs with a placeholder (optional)
    final_dataset[col] = final_dataset[col].fillna('missing_value')

    # Convert to category type first to manage the encoding
    final_dataset[col] = final_dataset[col].astype('category')
    
    # Factorize the column
    final_dataset[col] = pd.factorize(final_dataset[col])[0].astype('int64')

# Step 7: Restore original data types for other columns
final_dataset = final_dataset.astype(original_dtypes)

# Now, final_dataset contains all your data with missing values filled and original data types restored

train=final_dataset

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import VotingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import cohen_kappa_score
from scipy.optimize import minimize
from tqdm import tqdm
from IPython.display import clear_output

# Function definitions (unchanged)
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

def TrainML(model_class, test_data):
    X = train.drop(['sii'], axis=1)
    y = train['sii']

    # Ensure all categorical columns are encoded
    for col in categorical_features:
        X[col] = pd.factorize(X[col])[0].astype('int64')
    
    test_data = test_data.copy()
    for col in categorical_features:
        test_data[col] = pd.factorize(test_data[col])[0].astype('int64')

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {tKappa:.3f}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })

    return submission

# Model parameters and instantiation (unchanged)
Params = {
    'learning_rate': 0.046,
    'max_depth': 12,
    'num_leaves': 478,
    'min_data_in_leaf': 13,
    'feature_fraction': 0.893,
    'bagging_fraction': 0.784,
    'bagging_freq': 4,
    'lambda_l1': 10,
    'lambda_l2': 0.01
}

XGB_Params = {
    'learning_rate': 0.05,
    'max_depth': 6,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 1,
    'reg_lambda': 5,
    'random_state': SEED
}

CatBoost_Params = {
    'learning_rate': 0.05,
    'depth': 6,
    'iterations': 200,
    'random_state': SEED,
    'cat_features': cat_c,
    'verbose': 0,
    'l2_leaf_reg': 10
}

Light = LGBMRegressor(**Params, random_state=SEED, verbose=-1, n_estimators=300)
XGB_Model = XGBRegressor(**XGB_Params)
CatBoost_Model = CatBoostRegressor(**CatBoost_Params)

# Combine models using Voting Regressor
voting_model = VotingRegressor(estimators=[
    ('lightgbm', Light),
    ('xgboost', XGB_Model),
    ('catboost', CatBoost_Model)
])

# Train the ensemble model
Submission = TrainML(voting_model, test)

# Save submission
# Submission.to_csv('submission.csv', index=False)

 59%|█████▉    | 591/996 [04:18<02:09,  3.12it/s]

In [ ]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

from colorama import Fore, Style
from IPython.display import clear_output
import warnings
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

SEED = 42
n_splits = 5

# Load datasets
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')




def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]



def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
    stats, indexes = zip(*results)
    
    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    return df

        
train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

time_series_cols = train_ts.columns.tolist()
time_series_cols.remove("id")

train = pd.merge(train, train_ts, how="left", on='id')
test = pd.merge(test, test_ts, how="left", on='id')

train = train.drop('id', axis=1)
test = test.drop('id', axis=1)   


##########

# import pandas as pd
# from sklearn.impute import KNNImputer

# # Assuming 'train' is your DataFrame

# # Step 1: Create the KNN imputer
# imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors

# # Step 2: Select numeric columns and fit the imputer
# numeric_cols = train.select_dtypes(include=['float64', 'int64']).columns
# imputed_data = imputer.fit_transform(train[numeric_cols])

# # Step 3: Create a new DataFrame with the imputed values
# train_imputed = pd.DataFrame(imputed_data, columns=numeric_cols)

# # Step 4: Convert the 'sii' column back to integers
# train_imputed['sii'] = train_imputed['sii'].round().astype(int)

# # If there are other columns to retain, you can merge them back
# for col in train.columns:
#     if col not in numeric_cols:
#         train_imputed[col] = train[col]

# # Now, check if 'sii' has been filled and is of integer type
# print(train_imputed['sii'].isna().sum())  # Should be 0 if all NaNs were filled
# print(train_imputed['sii'].dtype)  # Should show 'int'

# train_imputed['sii']=train['sii']

# train=train_imputed


#########


featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
                'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
                'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
                'PreInt_EduHx-computerinternet_hoursday', 'sii']

featuresCols += time_series_cols

train = train[featuresCols]
train = train.dropna(subset='sii')

cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 
          'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
          'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']

def update(df):
    global cat_c
    for c in cat_c: 
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
    return df
        
train = update(train)
test = update(test)

def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
    return {value: idx for idx, value in enumerate(unique_values)}

for col in cat_c:
    mapping = create_mapping(col, train)
    mappingTe = create_mapping(col, test)
    
    train[col] = train[col].replace(mapping).astype(int)
    test[col] = test[col].replace(mappingTe).astype(int)

def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

def TrainML(model_class, test_data):
    X = train.drop(['sii'], axis=1)
    y = train['sii']

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })

    return submission

# Model parameters for LightGBM
Params = {
    'learning_rate': 0.046,
    'max_depth': 12,
    'num_leaves': 478,
    'min_data_in_leaf': 13,
    'feature_fraction': 0.893,
    'bagging_fraction': 0.784,
    'bagging_freq': 4,
    'lambda_l1': 10,  # Increased from 6.59
    'lambda_l2': 0.01  # Increased from 2.68e-06
}


# XGBoost parameters
XGB_Params = {
    'learning_rate': 0.05,
    'max_depth': 6,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 1,  # Increased from 0.1
    'reg_lambda': 5,  # Increased from 1
    'random_state': SEED
}


CatBoost_Params = {
    'learning_rate': 0.05,
    'depth': 6,
    'iterations': 200,
    'random_state': SEED,
    'cat_features': cat_c,
    'verbose': 0,
    'l2_leaf_reg': 10  # Increase this value
}

# Create model instances
Light = LGBMRegressor(**Params, random_state=SEED, verbose=-1, n_estimators=300)
XGB_Model = XGBRegressor(**XGB_Params)
CatBoost_Model = CatBoostRegressor(**CatBoost_Params)

# Combine models using Voting Regressor
voting_model = VotingRegressor(estimators=[
    ('lightgbm', Light),
    ('xgboost', XGB_Model),
    ('catboost', CatBoost_Model)
])

# Train the ensemble model
Submission1 = TrainML(voting_model, test)

# Save submission
#Submission1.to_csv('submission.csv', index=False)

In [ ]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

from colorama import Fore, Style
from IPython.display import clear_output
import warnings
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

SEED = 42
n_splits = 5

# Load datasets
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    stats, indexes = zip(*results)
    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    return df

train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

time_series_cols = train_ts.columns.tolist()
time_series_cols.remove("id")

train = pd.merge(train, train_ts, how="left", on='id')
test = pd.merge(test, test_ts, how="left", on='id')

train = train.drop('id', axis=1)
test = test.drop('id', axis=1)


#####


# import pandas as pd
# from sklearn.impute import KNNImputer

# # Assuming 'train' is your DataFrame

# # Step 1: Create the KNN imputer
# imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors

# # Step 2: Select numeric columns and fit the imputer
# numeric_cols = train.select_dtypes(include=['float64', 'int64']).columns
# imputed_data = imputer.fit_transform(train[numeric_cols])

# # Step 3: Create a new DataFrame with the imputed values
# train_imputed = pd.DataFrame(imputed_data, columns=numeric_cols)

# # Step 4: Convert the 'sii' column back to integers
# train_imputed['sii'] = train_imputed['sii'].round().astype(int)

# # If there are other columns to retain, you can merge them back
# for col in train.columns:
#     if col not in numeric_cols:
#         train_imputed[col] = train[col]

# # Now, check if 'sii' has been filled and is of integer type
# print(train_imputed['sii'].isna().sum())  # Should be 0 if all NaNs were filled
# print(train_imputed['sii'].dtype)  # Should show 'int'

# train_imputed['sii']=train['sii']

# train=train_imputed

#####

featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-Season', 'CGAS-CGAS_Score',
                'Physical-Season', 'Physical-BMI', 'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP', 'Fitness_Endurance-Season',
                'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec', 'FGC-Season',
                'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone',
                'FGC-FGC_PU', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone',
                'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat',
                'BIA-BIA_Frame_num', 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM', 'BIA-BIA_TBW', 'PAQ_A-Season',
                'PAQ_A-PAQ_A_Total', 'PAQ_C-Season', 'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T',
                'PreInt_EduHx-Season', 'PreInt_EduHx-computerinternet_hoursday', 'sii']

featuresCols += time_series_cols

train = train[featuresCols]
train = train.dropna(subset='sii')

cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
         'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']

def update(df):
    global cat_c
    for c in cat_c: 
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
    return df

train = update(train)
test = update(test)

def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
    return {value: idx for idx, value in enumerate(unique_values)}

for col in cat_c:
    mapping = create_mapping(col, train)
    mappingTe = create_mapping(col, test)
    
    train[col] = train[col].replace(mapping).astype(int)
    test[col] = test[col].replace(mappingTe).astype(int)

def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

def TrainML(model_class, test_data):
    X = train.drop(['sii'], axis=1)
    y = train['sii']

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tp_rounded = threshold_Rounder(tpm, KappaOPtimizer.x)

    return tp_rounded

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Imputation step: Filling missing values with the median
imputer = SimpleImputer(strategy='median')

# Updating the ensemble to include the RandomForest and GradientBoosting models
ensemble = VotingRegressor(estimators=[
    ('lgb', Pipeline(steps=[('imputer', imputer), ('regressor', LGBMRegressor(random_state=SEED))])),
    ('xgb', Pipeline(steps=[('imputer', imputer), ('regressor', XGBRegressor(random_state=SEED))])),
    ('cat', Pipeline(steps=[('imputer', imputer), ('regressor', CatBoostRegressor(random_state=SEED, silent=True))])),
    ('rf', Pipeline(steps=[('imputer', imputer), ('regressor', RandomForestRegressor(random_state=SEED))])),
    ('gb', Pipeline(steps=[('imputer', imputer), ('regressor', GradientBoostingRegressor(random_state=SEED))]))
])

# Train the ensemble with the updated model pipeline
predictions = TrainML(ensemble, test)

# Save predictions to a CSV file
sample['sii'] = predictions
#sample.to_csv('submission.csv', index=False)

In [5]:
# ==============================================
# Import Necessary Libraries
# ==============================================
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings

from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedShuffleSplit
from scipy.optimize import minimize
from sklearn.impute import KNNImputer

from scipy.stats import mode  # For majority voting

# PyTorch Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Suppress warnings and set display options
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

# Set random seed for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# ==============================================
# Load and Merge Data
# ==============================================
# Load main datasets
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')

# ==============================================
# Feature Engineering
# ==============================================
def engineer_features(df):
    """
    Create interaction features.
    """
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['Pulse_Pressure'] = df['Physical-Systolic_BP'] - df['Physical-Diastolic_BP']
    df['HeartRate_Age'] = df['Physical-HeartRate'] * df['Basic_Demos-Age']
    df['Fitness_Score'] = df['Fitness_Endurance-Max_Stage'] * (
        df['Fitness_Endurance-Time_Mins'] * 60 + df['Fitness_Endurance-Time_Sec']
    )
    df['FMI_FFMI_Ratio'] = df['BIA-BIA_FMI'] / (df['BIA-BIA_FFMI'] + 1e-6)
    df['Sleep_Internet_Hours'] = df['SDS-SDS_Total_T'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['Waist_Height_Ratio'] = df['Physical-Waist_Circumference'] / (df['Physical-Height'] + 1e-6)
    return df

# Apply feature engineering to both train and test datasets
train = engineer_features(train)
test = engineer_features(test)

# ==============================================
# Define Feature Columns
# ==============================================
featuresCols = list(set(train.columns).intersection(set(test.columns)))
# featuresCols = [
#     'Internet_Hours_Age', 'Physical-Height', 'Sleep_Internet_Hours', 'Basic_Demos-Age', 
#     'BMI_Internet_Hours', 'HeartRate_Age', 'BMI_Age', 'Physical-Waist_Circumference', 
#     'FGC-FGC_CU', 'BIA-BIA_BMI', 'SDS-SDS_Total_Raw', 'FGC-FGC_GSND', 'Physical-BMI', 
#     'FGC-FGC_GSD', 'FGC-FGC_PU', 'BIA-BIA_FFMI', 'BIA-BIA_Frame_num', 'Physical-Systolic_BP', 
#     'Pulse_Pressure', 'FGC-FGC_TL', 'Basic_Demos-Sex', 'FGC-FGC_SRL_Zone', 'CGAS-CGAS_Score', 
#     'Fitness_Score', 'BIA-BIA_SMM', 'BIA-BIA_Activity_Level_num'
# ]

# Select the features from train and test datasets
train_df = train[featuresCols + ['sii']]  # Include 'sii' in train features
test_df = test[featuresCols]

# Drop 'id' from train and test data if present
train_df = train_df.drop('id', axis=1, errors='ignore')
test_df = test_df.drop('id', axis=1, errors='ignore')

# Drop rows with missing target variable 'sii' in train data
train_df = train_df.dropna(subset=['sii'])

featuresCols = list(test_df.columns)

# ==============================================
# Handle Categorical Variables
# ==============================================
# List of categorical columns
cat_c = [
    'Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 
    'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
    'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season'
]

def create_consistent_mapping(column):
    """
    Create a consistent mapping for categorical variables by combining unique values from both datasets.
    """
    combined_values = pd.concat([train_df[column], test_df[column]], axis=0).fillna('Missing')
    unique_values = combined_values.unique()
    sorted_values = np.sort(unique_values)  # Ensure consistent ordering
    return {value: idx for idx, value in enumerate(sorted_values)}

# Map categorical variables to integers using a consistent mapping
for col in cat_c:
    mapping = create_consistent_mapping(col)
    train_df[col] = train_df[col].fillna('Missing').map(mapping).astype(int)
    test_df[col] = test_df[col].fillna('Missing').map(mapping).astype(int)


# ==============================================
# Handle Missing Values with KNN Imputer
# ==============================================
# Impute missing values using KNNImputer
imputer = KNNImputer(n_neighbors=3)
numeric_cols = train_df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Remove 'sii' from numeric_cols when applying to test data
if 'sii' in numeric_cols:
    numeric_cols.remove('sii')

# Impute train data
train_df[numeric_cols] = imputer.fit_transform(train_df[numeric_cols])

# Impute test data
# Ensure that all columns in numeric_cols exist in test data
numeric_cols_test = [col for col in numeric_cols if col in test_df.columns]
test_df[numeric_cols_test] = imputer.transform(test_df[numeric_cols_test])

# Ensure 'sii' remains integer
train_df['sii'] = train_df['sii'].round().astype(int)

# ==============================================
# Model Training and Evaluation Functions
# ==============================================
def quadratic_weighted_kappa(y_true, y_pred):
    """
    Calculate Quadratic Weighted Kappa.
    """
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    """
    Apply thresholds to continuous predictions.
    """
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    """
    Objective function for optimizing thresholds.
    """
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

# ==============================================
# Split Features Among Expert Networks
# ==============================================
feature_splits = np.array_split(featuresCols, 4)
features_expert1 = feature_splits[0].tolist()
features_expert2 = feature_splits[1].tolist()
features_expert3 = feature_splits[2].tolist()
features_expert4 = feature_splits[3].tolist()

# ==============================================
# Prepare Data for Training
# ==============================================
# Inputs for expert networks
X_expert1 = train_df[features_expert1]
X_expert2 = train_df[features_expert2]
X_expert3 = train_df[features_expert3]
X_expert4 = train_df[features_expert4]

# Input for weighting network
X_weighting = train_df[featuresCols]  # All features

# Target variable
y = train_df['sii']

# Stratified Split
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
train_index, val_index = next(sss.split(X_weighting, y))

X_expert_list_train = [X.iloc[train_index].reset_index(drop=True) for X in [X_expert1, X_expert2, X_expert3, X_expert4]]
X_weighting_train = X_weighting.iloc[train_index].reset_index(drop=True)
y_train = y.iloc[train_index].reset_index(drop=True)

X_expert_list_val = [X.iloc[val_index].reset_index(drop=True) for X in [X_expert1, X_expert2, X_expert3, X_expert4]]
X_weighting_val = X_weighting.iloc[val_index].reset_index(drop=True)
y_val = y.iloc[val_index].reset_index(drop=True)

# ==============================================
# Define Custom Dataset Class
# ==============================================
class MultiInputDataset(Dataset):
    def __init__(self, X_expert_list, X_weighting, y=None):
        self.X_expert_list = [torch.tensor(X.values, dtype=torch.float32) for X in X_expert_list]
        self.X_weighting = torch.tensor(X_weighting.values, dtype=torch.float32)
        if y is not None:
            self.y = torch.tensor(y.values, dtype=torch.float32)
        else:
            self.y = None
    def __len__(self):
        return len(self.X_weighting)
    def __getitem__(self, idx):
        inputs = [X[idx] for X in self.X_expert_list]
        weighting_input = self.X_weighting[idx]
        if self.y is not None:
            return inputs, weighting_input, self.y[idx]
        else:
            return inputs, weighting_input

# Create datasets and data loaders
train_dataset = MultiInputDataset(X_expert_list_train, X_weighting_train, y_train)
val_dataset = MultiInputDataset(X_expert_list_val, X_weighting_val, y_val)

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Prepare test data
X_expert1_test = test_df[features_expert1]
X_expert2_test = test_df[features_expert2]
X_expert3_test = test_df[features_expert3]
X_expert4_test = test_df[features_expert4]
X_weighting_test = test_df[featuresCols]

test_dataset = MultiInputDataset(
    [X_expert1_test.reset_index(drop=True), X_expert2_test.reset_index(drop=True), 
     X_expert3_test.reset_index(drop=True), X_expert4_test.reset_index(drop=True)],
    X_weighting_test.reset_index(drop=True)
)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# ==============================================
# Define Neural Network Architectures
# ==============================================
class ExpertNet(nn.Module):
    def __init__(self, input_size):
        super(ExpertNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Linear(32, 1)
        )
    def forward(self, x):
        return self.fc(x).squeeze()  # Output shape: [batch_size]

class WeightingNet(nn.Module):
    def __init__(self, input_size):
        super(WeightingNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Linear(32, 4),
            nn.Softmax(dim=1)  # Output weights that sum to 1
        )
    def forward(self, x):
        return self.fc(x)  # Output shape: [batch_size, 4]

# Initialize networks
input_sizes = [len(features_expert1), len(features_expert2), len(features_expert3), len(features_expert4)]
expert_nets = [ExpertNet(input_size) for input_size in input_sizes]
weighting_net = WeightingNet(len(featuresCols))

# Move models to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for net in expert_nets:
    net.to(device)
weighting_net.to(device)

# ==============================================
# Define Loss Function and Optimizer
# ==============================================
class OrdinalLoss(nn.Module):
    def __init__(self):
        super(OrdinalLoss, self).__init__()
    def forward(self, preds, targets):
        # preds: continuous predictions [batch_size]
        # targets: true labels [batch_size]
        # Compute loss that penalizes deviations more severely for larger errors
        loss = torch.mean((preds - targets.float()) ** 2)
        return loss

criterion = OrdinalLoss()

optimizer = optim.AdamW(
    list(expert_nets[0].parameters()) +
    list(expert_nets[1].parameters()) +
    list(expert_nets[2].parameters()) +
    list(expert_nets[3].parameters()) +
    list(weighting_net.parameters())
)

# ==============================================
# Training Loop
# ==============================================
num_epochs = 350

for epoch in range(num_epochs):
    expert_nets[0].train()
    expert_nets[1].train()
    expert_nets[2].train()
    expert_nets[3].train()
    weighting_net.train()
    
    running_loss = 0.0
    for inputs_list, weighting_input, labels in train_loader:
        optimizer.zero_grad()
        
        # Move data to device
        inputs_list = [inp.to(device) for inp in inputs_list]
        weighting_input = weighting_input.to(device)
        labels = labels.to(device)
        
        # Get outputs from expert networks
        expert_outputs = []
        for i in range(4):
            expert_outputs.append(expert_nets[i](inputs_list[i]))
        expert_outputs = torch.stack(expert_outputs, dim=1)  # Shape: [batch_size, 4]
        
        # Get weights from weighting network
        weights = weighting_net(weighting_input)  # Shape: [batch_size, 4]
        
        # Compute weighted sum
        preds = (expert_outputs * weights).sum(dim=1)  # Shape: [batch_size]
        
        # Compute loss
        loss = criterion(preds, labels)
        
        # Backpropagate
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * labels.size(0)
    epoch_loss = running_loss / len(train_dataset)
    
    # Validation
    expert_nets[0].eval()
    expert_nets[1].eval()
    expert_nets[2].eval()
    expert_nets[3].eval()
    weighting_net.eval()
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for inputs_list, weighting_input, labels in val_loader:
            # Move data to device
            inputs_list = [inp.to(device) for inp in inputs_list]
            weighting_input = weighting_input.to(device)
            labels = labels.to(device)
            
            # Get outputs from expert networks
            expert_outputs = []
            for i in range(4):
                expert_outputs.append(expert_nets[i](inputs_list[i]))
            expert_outputs = torch.stack(expert_outputs, dim=1)  # Shape: [batch_size, 4]
            
            # Get weights from weighting network
            weights = weighting_net(weighting_input)  # Shape: [batch_size, 4]
            
            # Compute weighted sum
            preds = (expert_outputs * weights).sum(dim=1)  # Shape: [batch_size]
            
            val_preds.append(preds.cpu().numpy())
            val_labels.append(labels.cpu().numpy())
    val_preds = np.concatenate(val_preds)
    val_labels = np.concatenate(val_labels)
    
    # Compute validation loss and QWK
    val_loss = np.mean((val_preds - val_labels) ** 2)
    val_preds_rounded = np.clip(np.round(val_preds), 0, 3).astype(int)
    val_kappa = quadratic_weighted_kappa(val_labels, val_preds_rounded)
    if val_kappa > 0.38:
        break
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val QWK: {val_kappa:.4f}")

# ==============================================
# Optimize Thresholds
# ==============================================
from scipy.optimize import minimize

# Optimize thresholds
KappaOptimizer = minimize(evaluate_predictions,
                          x0=[0.5, 1.5, 2.5], args=(val_labels, val_preds), 
                          method='Nelder-Mead')
best_thresholds = KappaOptimizer.x

# Evaluate optimized QWK
val_preds_tuned = threshold_Rounder(val_preds, best_thresholds)
val_kappa_tuned = quadratic_weighted_kappa(val_labels, val_preds_tuned)
print(f"Optimized Validation QWK: {val_kappa_tuned:.4f}")

# ==============================================
# Generate Predictions on Test Data
# ==============================================
expert_nets[0].eval()
expert_nets[1].eval()
expert_nets[2].eval()
expert_nets[3].eval()
weighting_net.eval()

test_preds = []
with torch.no_grad():
    for inputs_list, weighting_input in test_loader:
        # Move data to device
        inputs_list = [inp.to(device) for inp in inputs_list]
        weighting_input = weighting_input.to(device)
        
        # Get outputs from expert networks
        expert_outputs = []
        for i in range(4):
            expert_outputs.append(expert_nets[i](inputs_list[i]))
        expert_outputs = torch.stack(expert_outputs, dim=1)  # Shape: [batch_size, 4]
        
        # Get weights from weighting network
        weights = weighting_net(weighting_input)  # Shape: [batch_size, 4]
        
        # Compute weighted sum
        preds = (expert_outputs * weights).sum(dim=1)  # Shape: [batch_size]
        
        test_preds.append(preds.cpu().numpy())
test_preds = np.concatenate(test_preds)

# Apply optimized thresholds
test_preds_tuned = threshold_Rounder(test_preds, best_thresholds)
test_preds_tuned = test_preds_tuned.astype(int)

# Create submission DataFrame
submission4 = pd.DataFrame({
    'id': test['id'],
    'sii': test_preds_tuned
})

# Save submission
# submission4.to_csv('submission.csv', index=False)
# print("Submission file 'submission.csv' has been created.")

Epoch 1/350, Loss: 0.6882, Val Loss: 0.6590, Val QWK: 0.1572
Epoch 2/350, Loss: 0.5102, Val Loss: 0.5615, Val QWK: 0.2534
Epoch 3/350, Loss: 0.4958, Val Loss: 0.5065, Val QWK: 0.2922
Epoch 4/350, Loss: 0.4859, Val Loss: 0.5144, Val QWK: 0.2959
Epoch 5/350, Loss: 0.4651, Val Loss: 0.5259, Val QWK: 0.2486
Epoch 6/350, Loss: 0.4705, Val Loss: 0.5472, Val QWK: 0.2881
Epoch 7/350, Loss: 0.4625, Val Loss: 0.5675, Val QWK: 0.2982
Epoch 8/350, Loss: 0.4542, Val Loss: 0.5168, Val QWK: 0.2884
Epoch 9/350, Loss: 0.4580, Val Loss: 0.5178, Val QWK: 0.3134
Epoch 10/350, Loss: 0.4506, Val Loss: 0.5047, Val QWK: 0.3279
Epoch 11/350, Loss: 0.4417, Val Loss: 0.4729, Val QWK: 0.3401
Epoch 12/350, Loss: 0.4359, Val Loss: 0.5015, Val QWK: 0.3059
Epoch 13/350, Loss: 0.4387, Val Loss: 0.4895, Val QWK: 0.3162
Epoch 14/350, Loss: 0.4365, Val Loss: 0.5335, Val QWK: 0.2994
Epoch 15/350, Loss: 0.4337, Val Loss: 0.5220, Val QWK: 0.3168
Epoch 16/350, Loss: 0.4393, Val Loss: 0.5056, Val QWK: 0.2937
Epoch 17/350, Los

In [ ]:
# ==============================================
# Import Necessary Libraries
# ==============================================
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.display import clear_output
import warnings

from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from scipy.optimize import minimize
from sklearn.impute import KNNImputer

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor

# Suppress warnings and set display options
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

# Set random seed and number of folds for cross-validation
SEED = 42
n_splits = 10

# ==============================================
# Load and Merge Data
# ==============================================
# Load main datasets
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

# ==============================================
# Feature Engineering
# ==============================================
def engineer_features(df):
    """
    Create interaction features.
    """
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['Pulse_Pressure'] = df['Physical-Systolic_BP'] - df['Physical-Diastolic_BP']
    df['HeartRate_Age'] = df['Physical-HeartRate'] * df['Basic_Demos-Age']
    df['Fitness_Score'] = df['Fitness_Endurance-Max_Stage'] * (
        df['Fitness_Endurance-Time_Mins'] * 60 + df['Fitness_Endurance-Time_Sec']
    )
    df['FMI_FFMI_Ratio'] = df['BIA-BIA_FMI'] / (df['BIA-BIA_FFMI'] + 1e-6)
    df['Sleep_Internet_Hours'] = df['SDS-SDS_Total_T'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['Waist_Height_Ratio'] = df['Physical-Waist_Circumference'] / (df['Physical-Height'] + 1e-6)
    return df

# Apply feature engineering to both train and test datasets
train = engineer_features(train)
test = engineer_features(test)

# ==============================================
# Define Feature Columns
# ==============================================
# Define featuresCols as the common columns between train and test
featuresCols = list(set(train.columns).intersection(set(test.columns)))

# Select the features from train and test datasets
train = train[featuresCols + ['sii']]  # Include 'sii' in train features
test = test[featuresCols]

# Drop 'id' from train and test data if present
train = train.drop('id', axis=1, errors='ignore')
test = test.drop('id', axis=1, errors='ignore')

# Drop rows with missing target variable 'sii' in train data
train = train.dropna(subset=['sii'])

# ==============================================
# Handle Categorical Variables
# ==============================================
# List of categorical columns
cat_c = [
    'Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 
    'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
    'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season'
]

def update_categorical(df):
    """
    Fill missing values and convert columns to categorical.
    """
    for c in cat_c: 
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
    return df

# Update categorical columns in both datasets
train = update_categorical(train)
test = update_categorical(test)

def create_mapping(column, dataset):
    """
    Create a mapping for categorical variables.
    """
    unique_values = dataset[column].unique()
    return {value: idx for idx, value in enumerate(unique_values)}

# Map categorical variables to integers
for col in cat_c:
    mapping_train = create_mapping(col, train)
    mapping_test = create_mapping(col, test)
    
    train[col] = train[col].replace(mapping_train).astype(int)
    test[col] = test[col].replace(mapping_test).astype(int)

# ==============================================
# Handle Missing Values with KNN Imputer
# ==============================================
# Impute missing values using KNNImputer
imputer = KNNImputer(n_neighbors=5)
numeric_cols = train.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Remove 'sii' from numeric_cols when applying to test data
if 'sii' in numeric_cols:
    numeric_cols.remove('sii')

# Impute train data
train[numeric_cols] = imputer.fit_transform(train[numeric_cols])

# Impute test data
# Ensure that all columns in numeric_cols exist in test data
numeric_cols_test = [col for col in numeric_cols if col in test.columns]
test[numeric_cols_test] = imputer.transform(test[numeric_cols_test])

# Ensure 'sii' remains integer
train['sii'] = train['sii'].round().astype(int)

# ==============================================
# Model Training and Evaluation Functions
# ==============================================
def quadratic_weighted_kappa(y_true, y_pred):
    """
    Calculate Quadratic Weighted Kappa.
    """
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    """
    Apply thresholds to continuous predictions.
    """
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    """
    Objective function for optimizing thresholds.
    """
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

def TrainML(model_class, test_data):
    """
    Train the model using Stratified K-Fold cross-validation and evaluate it.
    """
    X = train.drop(['sii'], axis=1)
    y = train['sii']

    # Ensure all data is numeric
    X = X.apply(pd.to_numeric, errors='coerce')
    test_data = test_data.apply(pd.to_numeric, errors='coerce')

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    # Optimize thresholds
    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {tKappa:.3f}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })

    return submission

# ==============================================
# Model Parameters and Instantiation
# ==============================================
# Define model parameters
LGBMParams = {
    'learning_rate': 0.05,
    'max_depth': 5,
    'num_leaves': 100,
    'min_data_in_leaf': 13,
    'feature_fraction': 0.893,
    'bagging_fraction': 0.784,
    'bagging_freq': 4,
    'lambda_l1': 10,
    'lambda_l2': 1,
    'random_state': SEED,
    'verbose': -1,
    'n_estimators': 100
}

XGB_Params = {
    'learning_rate': 0.05,
    'max_depth': 5,
    'n_estimators': 100,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 1,
    'reg_lambda': 10,
    'random_state': SEED
}

CatBoost_Params = {
    'learning_rate': 0.05,
    'depth': 5,
    'iterations': 100,
    'random_state': SEED,
    'verbose': 0,
    'l2_leaf_reg': 10
}

# Instantiate models
Light = LGBMRegressor(**LGBMParams)
XGB_Model = XGBRegressor(**XGB_Params)
CatBoost_Model = CatBoostRegressor(**CatBoost_Params)

# Combine models using Voting Regressor
voting_model = VotingRegressor(estimators=[
    ('lightgbm', Light),
    ('xgboost', XGB_Model),
    ('catboost', CatBoost_Model)
])

# ==============================================
# Train Model and Generate Submission
# ==============================================
# Train the ensemble model
submission5 = TrainML(voting_model, test)

# Save submission
# submission5.to_csv('submission.csv', index=False)
# print("Submission file 'submission.csv' has been created.")

In [ ]:
import pandas as pd

# Load your three submission files
sub1 = Submission
sub2 = Submission1
sub3 = sample
sub4 = submission4
sub5 = submission5

# Ensure the IDs are aligned (if not sorted)
sub1 = sub1.sort_values(by='id').reset_index(drop=True)
sub2 = sub2.sort_values(by='id').reset_index(drop=True)
sub3 = sub3.sort_values(by='id').reset_index(drop=True)
sub4 = sub4.sort_values(by='id').reset_index(drop=True)
sub5 = sub5.sort_values(by='id').reset_index(drop=True)

# Combine the three predictions
combined = pd.DataFrame({
    'id': sub1['id'],
    'sii_1': sub1['sii'],
    'sii_2': sub2['sii'],
    'sii_3': sub3['sii'],
    'sii_4': sub4['sii'],
    'sii_5': sub5['sii']
})

# Apply majority voting
def majority_vote(row):
    return row.mode()[0]  # Mode gets the most frequent value (majority vote)

combined['final_sii'] = combined[['sii_1', 'sii_2', 'sii_3', 'sii_4', 'sii_5']].apply(majority_vote, axis=1)

# Create the final submission DataFrame
final_submission = combined[['id', 'final_sii']].rename(columns={'final_sii': 'sii'})

# Save the final submission to a CSV file
final_submission.to_csv('submission.csv', index=False)

print("Majority voting completed and saved to 'submission.csv'")

In [ ]:
final_submission